In [1]:
import os, argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from modules.model import Model
from modules.loss import MDNLoss
import hparams
from text import *
from utils.utils import *
from utils.writer import get_writer
from torch.utils.tensorboard import SummaryWriter
import math
import matplotlib.pyplot as plt


os.environ["CUDA_VISIBLE_DEVICES"]='0, 1'
torch.manual_seed(hparams.seed)
torch.cuda.manual_seed(hparams.seed)

train_loader, val_loader, collate_fn = prepare_dataloaders(hparams, stage=2)

checkpoint_path = f"training_log/aligntts/stage1/checkpoint_40000"
state_dict = {}
for k, v in torch.load(checkpoint_path)['state_dict'].items():
    state_dict[k[7:]]=v

model = Model(hparams).cuda()
model.load_state_dict(state_dict)
model = nn.DataParallel(model).cuda()

criterion = MDNLoss()
writer = get_writer(hparams.output_directory, f'{hparams.log_directory}/stage2')
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-4,
                             betas=(0.9, 0.98),
                             eps=1e-09)


def validate(model, criterion, val_loader, iteration, writer):
    model.eval()
    with torch.no_grad():
        n_data, val_loss = 0, 0
        for i, batch in enumerate(val_loader):
            n_data += len(batch[0])
            text_padded, mel_padded, align_padded, text_lengths, mel_lengths = [
                reorder_batch(x, hparams.n_gpus).cuda() for x in batch
            ]
            mel_padded = (mel_padded - torch.min(mel_padded))\
                         / torch.max((mel_padded - torch.min(mel_padded)))

            encoder_input = model.module.Prenet(text_padded)
            hidden_states, _ = model.module.FFT_lower(encoder_input, text_lengths)
            mu_sigma = model.module.get_mu_sigma(hidden_states)
            mdn_loss, log_prob_matrix = criterion(mu_sigma, mel_padded, text_lengths, mel_lengths)
            align = model.module.viterbi(log_prob_matrix, text_lengths, mel_lengths)
            
            mel_out = model.module.get_melspec(hidden_states, align, mel_lengths)
            fft_loss = nn.L1Loss()(mel_out, mel_padded)
            
            total_loss = mdn_loss + fft_loss
            val_loss += total_loss.item() * len(batch[0])

        val_loss /= n_data

    print(f"Validation Loss: {val_loss}")
    plt.figure(figsize=(12,3))
    plt.imshow(align[0].detach().cpu().t(), aspect='auto', origin='lower')
    plt.show()

    plt.figure(figsize=(12,3))
    plt.imshow(mel_padded[0].detach().cpu(), aspect='auto', origin='lower')
    plt.show()

    plt.figure(figsize=(12,3))
    plt.imshow(mel_out[0].detach().cpu(), aspect='auto', origin='lower')
    plt.show()
    
    '''
    writer.add_losses(total_loss.item(), iteration//hparams.accumulation, 'Validation')
    
    writer.add_specs(mel_padded.detach().cpu(),
                     mel_out.detach().cpu(),
                     mel_lengths.detach().cpu(),
                     iteration//hparams.accumulation, 'Validation')
    
    writer.add_alignments(probable_path.detach().cpu(),
                          text_lengths.detach().cpu(),
                          mel_lengths.detach().cpu(),
                          iteration//hparams.accumulation, 'Validation')
    '''
            
    model.train()

In [ ]:
iteration, loss = 0, 0
model.train()

print("Training Start!!!")

while True:
    for i, batch in enumerate(train_loader):
        text_padded, mel_padded, align_padded, text_lengths, mel_lengths = [
            reorder_batch(x, hparams.n_gpus).cuda() for x in batch
        ]
        mel_padded = (mel_padded - torch.min(mel_padded))\
                     / torch.max((mel_padded - torch.min(mel_padded)))

        total_loss = model(text_padded,
                           mel_padded,
                           align_padded,
                           text_lengths,
                           mel_lengths,
                           criterion,
                           stage=2)
        sub_loss = total_loss.mean()/hparams.accumulation
        sub_loss.backward()
        loss = loss+sub_loss.item()
        iteration += 1
        
        if iteration%hparams.accumulation == 0:
            # lr_scheduling(optimizer, iteration//hparams.accumulation) -> 1e-4 fix
            nn.utils.clip_grad_norm_(model.parameters(), hparams.grad_clip_thresh)
            optimizer.step()
            model.zero_grad()
            writer.add_losses(loss, iteration//hparams.accumulation, 'Train')
            loss=0

        if iteration%(hparams.iters_per_validation*hparams.accumulation)==0:
            validate(model, criterion, val_loader, iteration, writer)

        if iteration%(hparams.iters_per_checkpoint*hparams.accumulation)==0:
            save_checkpoint(model,
                            optimizer,
                            hparams.lr,
                            iteration//hparams.accumulation,
                            filepath=f'{hparams.output_directory}/{hparams.log_directory}/stage2')

        if iteration==(hparams.train_steps[2]*hparams.accumulation):
            break
            
    if iteration==(hparams.train_steps[2]*hparams.accumulation):
            break

Training Start!!!


/home/ubuntu/anaconda3/envs/pytorch_p36/AlignTTS/utils/utils.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  ids = lengths.new_tensor(torch.arange(0, max_len))
/home/ubuntu/anaconda3/envs/pytorch_p36/AlignTTS/modules/model.py:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  indices = path.new_tensor(torch.arange(path.max()+1).view(1,1,-1)) # 1, 1, L
/home/ubuntu/anaconda3/envs/pytorch_p36/AlignTTS/modules/model.py:176: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  align = 1.0*(path.new_tensor(indices==path.unsqueeze(